In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from xgboost import XGBClassifier

In [2]:
# data = pd.read_csv("csv/walk.csv",index_col=False)
# X = data[["r_hip_ang", "r_knee_ang", "r_ankle_ang", "r_elbow_ang", "r_shoulder_ang",]]
#         # "r_hip_vel", "r_knee_vel", "r_ankle_vel", "r_elbow_vel", "r_shoulder_vel",
#         # "r_hip_accel", "r_knee_accel", "r_ankle_accel", "r_elbow_accel", "r_shoulder_accel"]]
# # X = data.drop(['gender', "id"], axis=1)

# encode = {'male': 0, 'female': 1}
# y = data['gender'].map(encode)
# X  = preprocessing.scale(X)


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [71]:
data = pd.read_csv("csv/walk.csv",index_col=False)
encode = {'<40': 0, '>40': 1}
# encode = {'male': 0, 'female': 1}

X = data.loc[data['id'].isin([2,3,4,5,7,9])]
# X_train = X [["r_hip_ang", "r_knee_ang", "r_ankle_ang", "r_elbow_ang", "r_shoulder_ang"]]
#         "r_hip_vel", "r_knee_vel", "r_ankle_vel", "r_elbow_vel", "r_shoulder_vel",
#         "r_hip_accel", "r_knee_accel", "r_ankle_accel", "r_elbow_accel", "r_shoulder_accel"]]
X_train = X.drop(['id','gender','age'],axis=1) #,'r_shoulder_ang','r_elbow_ang','r_knee_vel_3','r_elbow_vel_3','r_shoulder_vel_3'],axis=1)
X_train = preprocessing.scale(X_train)
y_train = X['age'].map(encode)

X2 = data.loc[data['id'].isin([1,10])] 
# X_test = X2 [["r_hip_ang", "r_knee_ang", "r_ankle_ang", "r_elbow_ang", "r_shoulder_ang"]]
        # "r_hip_vel", "r_knee_vel", "r_ankle_vel", "r_elbow_vel", "r_shoulder_vel",
        # "r_hip_accel", "r_knee_accel", "r_ankle_accel", "r_elbow_accel", "r_shoulder_accel"]]
X_test = X2.drop(['id','gender','age'],axis=1) #,'r_shoulder_ang','r_elbow_ang','r_knee_vel_3','r_elbow_vel_3','r_shoulder_vel_3'],axis=1)
X_test = preprocessing.scale(X_test)
y_test = X2['age'].map(encode)

In [169]:
clf = XGBClassifier(
#BEST for Train: 2,3,4,5,7,9 and Test 1,10 Age
 reg_lambda = 0.5,
 reg_alpha = 0.5,
 n_estimators = 200,
 learning_rate = 0.1,
 gamma = 0,
 booster = 'gbtree',
 base_score = 0.9,
 min_child_weight = 1,
 max_depth = 6,
 subsample = 0.3,
 colsample_bytree = 0.4)
 #BEST for Train 1 2 5, Test 3 4 Gender
#  clf = XGBClassifier(reg_lambda = 0.5, 
#  reg_alpha = 1,
#  n_estimators = 200,
#  learning_rate = 0.1,
#  gamma = 0,
#  booster = 'gbtree',
#  base_score = 0.5,
#  min_child_weight = 1,
#  max_depth = 6,
#  subsample = 0.5,
#  colsample_bytree = 0.4)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


In [170]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

Accuracy: 0.7673436905297395
[[11003  2788]
 [ 3936 11174]]
              precision    recall  f1-score   support

           0       0.74      0.80      0.77     13791
           1       0.80      0.74      0.77     15110

    accuracy                           0.77     28901
   macro avg       0.77      0.77      0.77     28901
weighted avg       0.77      0.77      0.77     28901



In [94]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

param = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01,0.05,0.1],
    'booster': ['gbtree', 'gblinear'],
    'gamma': [0, 0.5, 1],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [0.5, 1, 5],
    'base_score': [0.2, 0.5, 0.9],
    'max_depth': range(1,10,2),
    'min_child_weight': range(1,10,2), 
    'subsample' : [i/10.0 for i in range(5,10)],
    'colsample_bytree' :[i/10.0 for i in range(5,10)]
}

In [103]:
rnd_search = RandomizedSearchCV(XGBClassifier(), param_distributions=param,n_jobs=-1, verbose = 10, n_iter =10)
rnd_search.fit(X_train,y_train)
rnd_search.best_score_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


0.9511637977956019

In [104]:
rnd_search.best_params_

{'subsample': 0.7,
 'reg_lambda': 5,
 'reg_alpha': 0.5,
 'n_estimators': 200,
 'min_child_weight': 9,
 'max_depth': 9,
 'learning_rate': 0.1,
 'gamma': 0.5,
 'colsample_bytree': 0.5,
 'booster': 'gbtree',
 'base_score': 0.2}